#### Digital Job Program Business Analytics with SQL and Python Final Project - SuperFoodsMax

This notebook covers all codes used in the exploration of dataset provided by DJP course "Business Analytics with SQL and Python".

First thing first, import necessary packages and load the dataset. Then get a taste of the data.

In [53]:
import pandas as pd
import numpy as np

df_raw = pd.read_csv("./dataset_2019_2022.csv")
df_raw.head()

,customer_id,product_id,basket_id,loyalty,household_type,age_band,department,brand,commodity,store,price,transaction_date
0,15803,1131974,57266,Loyalist,1 adult with kids,19-24,Grocery,private,Baked bread/buns/rolls,374,0.99,5/10/2020
1,15803,1051516,57266,Loyalist,1 adult with kids,19-24,Produce,national,Vegetables - all others,374,0.70,24/10/2020
2,15803,967254,57266,Loyalist,1 adult with kids,19-24,Pharmaceutical,national,Cold and flu,374,1.68,18/10/2020
3,15803,1134222,57266,Loyalist,1 adult with kids,19-24,Grocery,private,Paper housewares,374,2.59,23/10/2020
4,15803,1003421,57266,Loyalist,1 adult with kids,19-24,Grocery,national,Soup,374,0.60,27/10/2020


Inspect dataframe dimensions.

In [54]:
df_raw.shape

(77750, 12)

Inspect dataframe info.

In [55]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77750 entries, 0 to 77749
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customer_id       77750 non-null  int64  
 1   product_id        77750 non-null  int64  
 2   basket_id         77750 non-null  int64  
 3   loyalty           77750 non-null  object 
 4   household_type    77750 non-null  object 
 5   age_band          77750 non-null  object 
 6   department        77750 non-null  object 
 7   brand             77750 non-null  object 
 8   commodity         77750 non-null  object 
 9   store             77750 non-null  int64  
 10  price             77750 non-null  float64
 11  transaction_date  77750 non-null  object 
dtypes: float64(1), int64(4), object(7)
memory usage: 7.1+ MB


Inspect unique values one by one - missing / corrupt data can appear in ways you least expect it.

In [56]:
for col in df_raw:
    print(col)
    print(df_raw[col].sort_values().unique())
    print("")

customer_id
[  15803   15811   15815 ... 2791100 2794900 2811700]

product_id
[   35082    74176   124385 ... 18184230 18185306 18203921]

basket_id
[  51964   51965   51966 ... 6809800 6810700 6811600]

loyalty
['First Time Buyer' 'Loyalist' 'Promiscuous']

household_type
['1 adult with kids' '2 adults with kids' '2 adults with no kids'
 'Single female' 'Single male']

age_band
['19-24' '25-34' '35-44' '45-54' '55-64' 'Greater than 65']

department
['Cosmetics' 'Deli' 'Floral' 'Grocery' 'Meat' 'Nutrition' 'Pastry'
 'Pharmaceutical' 'Produce' 'Salad Bar' 'Seafood']

brand
['national' 'private']

commodity
['Air care' 'Analgesics' 'Antacids' 'Apparel' 'Apples'
 'Audio/video products' 'Automotive products' 'Baby foods' 'Baby hbc'
 'Bacon' 'Bag snacks' 'Baked bread/buns/rolls' 'Baked sweet goods'
 'Bakery party trays' 'Baking' 'Baking mixes' 'Baking needs' 'Bath'
 'Bath tissues' 'Batteries' 'Beef' 'Beers/ales' 'Berries' 'Beverage'
 'Bird seed' 'Bleach' 'Books' 'Bottle deposits' 'Bread'
 '

For variable **age_band**, changing **Greater than 65** to **65+** make it more natural.

In [57]:
df_proc = df_raw.copy(deep=True)
df_proc["age_band"] = np.where(df_proc["age_band"] == "Greater than 65", "65+", df_proc["age_band"])
df_proc["age_band"].unique()

array(['19-24', '25-34', '35-44', '45-54', '65+', '55-64'], dtype=object)

Variable **price** is renamed as **sales**.

In [58]:
df_proc["sales"] = df_proc["price"]
df_proc.drop("price", axis = 1, inplace = True)

Now that we have processed the raw data, we transform dataframe into our preferred format to make graphs.
The date is not correctly parsed when data is ingested, so we do it manually here.

In [61]:
df_plot1 = df_proc.copy()
df_plot1[["transaction_day","transaction_month","transaction_year"]] = df_plot1["transaction_date"].str.split(pat="/", expand=True)

In [62]:
df_plot1.head()

,customer_id,product_id,basket_id,loyalty,household_type,age_band,department,brand,commodity,store,transaction_date,sales,transaction_day,transaction_month,transaction_year
0,15803,1131974,57266,Loyalist,1 adult with kids,19-24,Grocery,private,Baked bread/buns/rolls,374,5/10/2020,0.99,5,10,2020
1,15803,1051516,57266,Loyalist,1 adult with kids,19-24,Produce,national,Vegetables - all others,374,24/10/2020,0.70,24,10,2020
2,15803,967254,57266,Loyalist,1 adult with kids,19-24,Pharmaceutical,national,Cold and flu,374,18/10/2020,1.68,18,10,2020
3,15803,1134222,57266,Loyalist,1 adult with kids,19-24,Grocery,private,Paper housewares,374,23/10/2020,2.59,23,10,2020
4,15803,1003421,57266,Loyalist,1 adult with kids,19-24,Grocery,national,Soup,374,27/10/2020,0.60,27,10,2020


DDMMYYYY is too granular for our need, so we go with YYYYMM

In [63]:
df_plot1["transaction_yearmonth"] = df_plot1["transaction_month"] + "-" + df_plot1["transaction_year"]
df_plot1["transaction_yearmonth"] = pd.to_datetime(df_plot1['transaction_yearmonth']).dt.strftime('%Y-%m')


C:\Users\playe\AppData\Local\Temp\ipykernel_1792\1883118727.py:2: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



The resultant dataframe from this transformation shows a customer's spending in that month, should one have bought anything.

In [64]:
df_plot1 = df_plot1.groupby(by = ["transaction_yearmonth", "loyalty", "customer_id"])["sales"].sum().reset_index()
df_plot1.head()

,transaction_yearmonth,loyalty,customer_id,sales
0,2019-01,First Time Buyer,27872,44.18
1,2019-01,Loyalist,22115,75.77
2,2019-01,Loyalist,22317,105.69
3,2019-01,Loyalist,23057,74.51
4,2019-01,Loyalist,23876,60.62


Such spending is averaged across the same month and the same loyalty class, so the sales here represents average spending per customer in that month in a particular loyalty class.

In [65]:
df_plot1 = df_plot1.groupby(by = ["transaction_yearmonth", "loyalty"])["sales"].mean().reset_index()
df_plot1.head()

,transaction_yearmonth,loyalty,sales
0,2019-01,First Time Buyer,44.180000
1,2019-01,Loyalist,75.372105
2,2019-01,Promiscuous,70.586667
3,2019-02,First Time Buyer,9.990000
4,2019-02,Loyalist,69.360882


**Plotly** is used to produce graphs as it is both interactive and visually appealing.

In [66]:
import plotly.express as px

fig = px.line(df_plot1,
              x = "transaction_yearmonth",
              y = "sales",
              color = "loyalty",
              title = "Average sales per customer by customer loyalty over the past 3 years")
fig.show()

Now make another plot, but this time we wish to know total spending by a particular loyalty class in any given month.

In [67]:
df_plot2 = df_proc.copy()
df_plot2[["transaction_day","transaction_month","transaction_year"]] = df_plot2["transaction_date"].str.split(pat = "/", expand = True)
df_plot2["transaction_yearmonth"] = df_plot2["transaction_month"] + "-" + df_plot2["transaction_year"]
df_plot2["transaction_yearmonth"] = pd.to_datetime(df_plot2['transaction_yearmonth']).dt.strftime('%Y-%m')
df_plot2 = df_plot2.groupby(by=["transaction_yearmonth", "loyalty"])["sales"].sum().reset_index()
df_plot2.head()

C:\Users\playe\AppData\Local\Temp\ipykernel_1792\3458006447.py:4: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



,transaction_yearmonth,loyalty,sales
0,2019-01,First Time Buyer,44.18
1,2019-01,Loyalist,1432.07
2,2019-01,Promiscuous,3811.68
3,2019-02,First Time Buyer,9.99
4,2019-02,Loyalist,2358.27


In [68]:
fig = px.line(df_plot2,
              x = "transaction_yearmonth",
              y = "sales",
              color = "loyalty",
              title = "Total sales by customer loyalty over the past 3 years")
fig.show()

We can also mine the data to get frequently bought together items from this dataset - with the use of **mlxtend** package. Unfortunately the data does not come with **product_name** so **product_id** is used directly.

In [70]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

basket = pd.pivot_table(data = df_proc,
                        index = 'basket_id',
                        columns = 'product_id',
                        values = 'sales',
                        aggfunc = 'sum',
                        fill_value = 0)
basket.head()

product_id,35082,74176,124385,140705,216216,431392,522435,714433,721164,777857,...,18005913,18006037,18022633,18038459,18055202,18055205,18055532,18184230,18185306,18203921
basket_id,,,,,,,,,,,,,,,,,,,,,
51964,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
51965,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
51966,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
51967,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
51968,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0


If an item is bought, label it as 1 rather than having the amount spent in the sparse matrix.

In [71]:
for col in basket:
    basket[col] = np.where(basket[col] > 0, 1, 0)

In [76]:
basket

product_id,35082,74176,124385,140705,216216,431392,522435,714433,721164,777857,...,18005913,18006037,18022633,18038459,18055202,18055205,18055532,18184230,18185306,18203921
basket_id,,,,,,,,,,,,,,,,,,,,,
51964,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
51965,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
51966,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
51967,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
51968,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6806400,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6808800,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6809800,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Minimum support is set to 0.03 as setting the threshold too high will return nothing.

In [97]:
frequent_itemsets = apriori(basket, min_support = 0.03, use_colnames = True)

c:\Users\playe\miniconda3\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning:

DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type



What presents here is essentially the frequency of an item showing up in an order.

In [92]:
frequent_itemsets.head()

,support,itemsets
0,0.033448,(820165)
1,0.011435,(823704)
2,0.046598,(824005)
3,0.011149,(824180)
4,0.014866,(824813)


The threshold here refers to the minimum value **lift** can take. If two items are bought independently then lift will be 1, so it is apparent the lift will at least be 1 for a recommendation engine to be useful.

In [102]:
rules = association_rules(frequent_itemsets, metric = "lift", min_threshold = 1)
rules.shape

(18, 10)

So here are list of items (consequents) that should be recommended when the other one (antecedents) has been put in the basket using arbitrary thresholds set by us.

In [105]:
rules.sort_values("lift", ascending = False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(1082185),(844179),0.394225,0.074900,0.034019,0.086294,1.152129,0.004492,1.012471,0.217972
1,(844179),(1082185),0.074900,0.394225,0.034019,0.454198,1.152129,0.004492,1.109881,0.142732
11,(995242),(1082185),0.072613,0.394225,0.031732,0.437008,1.108523,0.003107,1.075991,0.105564
10,(1082185),(995242),0.394225,0.072613,0.031732,0.080493,1.108523,0.003107,1.008570,0.161609
8,(962568),(1082185),0.070326,0.394225,0.030589,0.434959,1.103327,0.002865,1.072091,0.100735
9,(1082185),(962568),0.394225,0.070326,0.030589,0.077592,1.103327,0.002865,1.007878,0.154596
4,(1082185),(866227),0.394225,0.108634,0.046884,0.118927,1.094752,0.004058,1.011683,0.142877
5,(866227),(1082185),0.108634,0.394225,0.046884,0.431579,1.094752,0.004058,1.065715,0.097099
14,(1082185),(1044078),0.394225,0.072327,0.030875,0.078318,1.082826,0.002362,1.006500,0.126269
15,(1044078),(1082185),0.072327,0.394225,0.030875,0.426877,1.082826,0.002362,1.056972,0.082454
